In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [5]:
df = pd.read_csv("trainingFeatures.csv")

In [6]:
df

,Unnamed: 0,displacementX,displacementY,displacement,totalTravelX,totalTravelY,totalTravel,timeElapsed,meanStepDisplacementX,meanStepDisplacementY,...,lab,meanVideoStepDisplacementX,meanVideoStepDisplacementY,meanVideoStepDisplacement,sdVideoMeanStepDisplacementX,sdVideoMeanStepDisplacementY,sdVideoMeanStepDisplacement,sdVideoMeanStepTravelX,sdVideoMeanStepTravelY,sdVideoMeanStepTravel
0,0,-37.353,-15.415,40.408772,40.603,24.497,47.420529,301.0,-0.124510,-0.051383,...,1.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
1,1,-27.248,-18.540,32.957322,33.082,28.292,43.529944,301.0,-0.090827,-0.061800,...,1.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
2,2,-29.438,-11.623,31.649486,38.020,26.041,46.083121,301.0,-0.098127,-0.038743,...,1.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
3,3,-39.837,-14.879,42.524948,46.421,34.359,57.753356,301.0,-0.132790,-0.049597,...,1.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
4,4,-1.616,0.055,1.616936,17.134,20.303,26.566629,301.0,-0.005387,0.000183,...,1.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16064,29,201.000,37.000,204.377102,557.000,509.000,754.539595,300.0,0.670000,0.123333,...,0.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781
16065,30,530.000,137.000,547.420314,548.000,311.000,630.099199,181.0,2.928177,0.756906,...,0.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781
16066,31,2.000,140.000,140.014285,564.000,602.000,824.924239,300.0,0.006667,0.466667,...,0.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781
16067,32,21.000,24.000,31.890437,21.000,24.000,31.890437,12.0,1.750000,2.000000,...,0.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781


In [7]:
df.sample(frac=1)
trainsize = int(len(df.index)*0.8)
df_train = df.iloc[:trainsize,:]
df_test = df.iloc[trainsize:,:]

In [8]:
df_train

,Unnamed: 0,displacementX,displacementY,displacement,totalTravelX,totalTravelY,totalTravel,timeElapsed,meanStepDisplacementX,meanStepDisplacementY,...,lab,meanVideoStepDisplacementX,meanVideoStepDisplacementY,meanVideoStepDisplacement,sdVideoMeanStepDisplacementX,sdVideoMeanStepDisplacementY,sdVideoMeanStepDisplacement,sdVideoMeanStepTravelX,sdVideoMeanStepTravelY,sdVideoMeanStepTravel
0,0,-37.353,-15.415,40.408772,40.603,24.497,47.420529,301.0,-0.124510,-0.051383,...,1.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.650310,0.958145,1.142472
1,1,-27.248,-18.540,32.957322,33.082,28.292,43.529944,301.0,-0.090827,-0.061800,...,1.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.650310,0.958145,1.142472
2,2,-29.438,-11.623,31.649486,38.020,26.041,46.083121,301.0,-0.098127,-0.038743,...,1.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.650310,0.958145,1.142472
3,3,-39.837,-14.879,42.524948,46.421,34.359,57.753356,301.0,-0.132790,-0.049597,...,1.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.650310,0.958145,1.142472
4,4,-1.616,0.055,1.616936,17.134,20.303,26.566629,301.0,-0.005387,0.000183,...,1.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.650310,0.958145,1.142472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12850,61,269.000,282.000,389.724261,269.000,282.000,389.724261,273.0,0.985348,1.032967,...,0.0,0.903781,1.127982,1.604478,0.706748,0.554957,0.567699,0.524893,0.571984,0.719275
12851,62,227.000,211.000,309.919344,227.000,211.000,309.919344,216.0,1.050926,0.976852,...,0.0,0.903781,1.127982,1.604478,0.706748,0.554957,0.567699,0.524893,0.571984,0.719275
12852,63,302.000,305.000,429.219058,302.000,305.000,429.219058,300.0,1.006667,1.016667,...,0.0,0.903781,1.127982,1.604478,0.706748,0.554957,0.567699,0.524893,0.571984,0.719275
12853,64,289.000,303.000,418.724253,289.000,303.000,418.724253,289.0,1.000000,1.048443,...,0.0,0.903781,1.127982,1.604478,0.706748,0.554957,0.567699,0.524893,0.571984,0.719275
